In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon, Point
import folium
from branca.colormap import StepColormap

# 1) 엑셀 읽기 & 컬럼명 단순화
df = pd.read_excel(
    r'D:\스터디\공공교육데이터\seoul-school-zone-analysis\data\additional\대화형_인구정책지도_2306_50m_전처리 완료.xlsx',
    engine='openpyxl'
).rename(columns={
    'geometry/coordinates/0/0/0/0': 'lon',
    'geometry/coordinates/0/0/0/1': 'lat',
    'properties/tot_popltn':      'pop'
})

# 2) Point GeoDataFrame (EPSG:4326)
gdf_pts = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df.lon, df.lat),
    crs='EPSG:4326'
)

# 3) WebMercator(3857) 으로 투영 → 미터 단위로 정사각형 생성
gdf_m = gdf_pts.to_crs(epsg=3857)
half = 25  # 50m 격자의 반변 길이

# 각 점마다 50m 정사각형 박스 생성
boxes = gdf_m.geometry.apply(
    lambda p: Polygon([
        (p.x-half, p.y-half),
        (p.x-half, p.y+half),
        (p.x+half, p.y+half),
        (p.x+half, p.y-half)
    ])
)
gdf_box_m = gpd.GeoDataFrame(
    gdf_m.drop(columns='geometry'),
    geometry=boxes,
    crs='EPSG:3857'
)

# 4) 다시 위경도(EPSG:4326)로 변환
gdf_box = gdf_box_m.to_crs(epsg=4326)

# 5) Folium 시각화 (나머지는 동일)
m = folium.Map(
    location=[gdf_box.geometry.centroid.y.mean(),
              gdf_box.geometry.centroid.x.mean()],
    zoom_start=13, tiles='cartodbpositron'
)

# 단계별 색상
pop = gdf_box['pop']
breaks = list(pop.quantile([0, .2, .4, .6, .8, 1.0]))
cmap = StepColormap(
    ['#f7fbff','#deebf7','#9ecae1','#6baed6','#3182bd','#08519c'],
    index=breaks,
    vmin=breaks[0], vmax=breaks[-1],
    caption='총인구'
)
cmap.add_to(m)

folium.GeoJson(
    gdf_box,
    style_function=lambda feat: {
        'fillColor': cmap(feat['properties']['pop']),
        'fillOpacity': 0.8,
        'weight': 0
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['pop'],
        aliases=['총인구:'],
        localize=True
    )
).add_to(m)

m  # Jupyter 노트북에서 바로 보이거나, m.save(...) 로 HTML 출력


FileNotFoundError: [Errno 2] No such file or directory: '.\\additional\\대화형_인구정책지도_2306_50m_전처리 완료.xlsx'